# Nested-CV 

Cet exemple compare les stratégies de validation croisée non imbriquée et imbriquée pour l'estimation des intervalles de prédiction avec `MapieRegressor`.

Dans la méthode séquentielle standard, une recherche de paramètres par validation croisée est effectuée sur l'ensemble du jeu d'entraînement. 
Le "meilleur modèle", le modèle avec le jeu de paramètres donnant le meilleur score, est ensuite utilisé dans MAPIE pour estimer les intervalles de confiance associés aux prédictions. 
Une limitation de cette méthode est que les résidus utilisés par MAPIE sont calculés sur l'ensemble du jeu de validation, qui avait été utilisé dans l'étape précédente pour optimiser le modèle via le réglage des hyperparamètres.
Cette méthode séquentielle conduit donc MAPIE à être légèrement trop optimiste avec les intervalles de confiance.

Pour résoudre ce problème, une autre option consiste à effectuer une recherche de paramètres par validation croisée imbriquée directement dans l'estimateur `MapieRegressor` afin de trouver le *meilleur* modèle perturbé pour son jeu de données *out-of-fold* d'entraînement correspondant. 
Cela garantit que les données du dernier pli servant à calculer les résidus estimés par MAPIE ne sont jamais vus par l'algorithme au préalable pour entraîner le modèle perturbé. Cependant, cette méthode est beaucoup plus lourde et augmente sensiblement le temps de calcul puisqu'elle entraîne $N \times P$ calculs, où $N$ est le nombre de modèles out-of-fold et $P$ le nombre de validations croisées de recherche de paramètres, contre $N + P$ pour l'approche non imbriquée.

Ici, nous comparons les deux stratégies sur le jeu de données *Boston dataset*. Nous utilisons le régresseur Random Forest comme régresseur de base pour la stratégie CV+. Pour alléger les calculs, nous adoptons une stratégie de recherche de paramètres `RandomizedSearchCV` avec un faible nombre d'itérations et un état aléatoire reproductible.

Les deux approches donnent des prédictions légèrement différentes, l'approche CV imbriquée estimant des largeurs d'intervalle de prédiction légèrement plus grandes de quelques pourcents au maximum (à part quelques exceptions).

Pour cet exemple, les deux approches donnent des scores similaires et des couvertures effectives identiques.

Dans le cas général, l'approche recommandée est d'utiliser la validation croisée emboîtée, car elle ne sous-estime pas les résidus et donc les intervalles de prédiction. Cependant, dans cet exemple particulier, les couvertures effectives des méthodes emboîtées et non emboîtées sont identiques.

## 1. Importation des données et définition du modèle

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_regression

from scipy.stats import randint
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

import plotly.graph_objects as go

**Exercice.** Commençons par importer `MapieRegressor et `coverage_score`.

In [ ]:
from mapie.estimators import MapieRegressor  # correction
from mapie.metrics import coverage_score  # correction

Choisissons ensuite les paramètres définissant la validation-croisée et l'estimation des intervalles de confiance.

In [ ]:
# Cross-validation and prediction-interval parameters.
test_size = 0.2
cv = 5
n_iter = 5
alpha = 0.05
method = "plus"
random_state = 42

**Exercice.** Importons le jeu de données Boston et séparons le en deux jeux de données d'entraînement et de test.

In [ ]:
# Load the Boston data
X_boston, y_boston = load_boston(return_X_y=True)

# Split the data into training and test sets.
X_train, X_test, y_train, y_test = train_test_split(
    X_boston, y_boston, test_size=test_size, random_state=random_state  # correction
)



**Exercice.** Définissons notre modèle `RandomForestRegressor` comme modèle de base et le domaine des paramètres exploré.

In [ ]:
# Define the Random Forest model as base regressor with parameter ranges.
rf_model = RandomForestRegressor(random_state=random_state, verbose=0)
rf_params = {
    "max_depth": randint(2, 30),
    "n_estimators": randint(10, 1e3)
}

## 2. Validation-croisée non-imbriquée

**Exercice.** Effectuons l'étude de paramètres par validation croisée. Pour ce faire, [`RandomizedSearchCV`](https://scikit-learn.org/stable/modules/grid_search.html#randomized-parameter-search) est uilisé. Il suffit de définir l'estimateur, la distribution des paramètres, le nombre d'itérations, et le nombre de splits. 

In [ ]:
cv_obj = RandomizedSearchCV(
    rf_model,  # correction
    param_distributions=rf_params,  # correction
    n_iter=n_iter,  # correction
    cv=cv,  # correction
    scoring="neg_root_mean_squared_error",
    verbose=0,
    random_state=random_state,
    n_jobs=-1,
)
cv_obj.fit(X_train, y_train)
best_est = cv_obj.best_estimator_

**Question.** Quel est le jeu de paramètres donnant le meilleur score ? Comment varie t-il avec la graine aléatoire ?

In [ ]:
print(cv_obj.best_params_)

**Exercice.** Implémentons maintenant l'estimateur de base déterminé par l'étude de paramètres dans `MapieRegressor` pour estimer les intervalles de prédiction associés. Il suffit ici de déterminer l'estimateur, la méthode ("plus"), et le nombre de plis pour la validation croisée. 

In [ ]:
mapie_non_nested = MapieRegressor(
    best_est,  # correction
    method="plus",  # correction
    cv=cv,  # correction
    ensemble=True,
    n_jobs=-1
)
mapie_non_nested.fit(X_train, y_train);

**Exercice.** Calculons les intervalles de prédiction sur `X_test` en définissant le paramètre `alpha`. 

In [ ]:
y_preds_non_nested = mapie_non_nested.predict(
    X_test, alpha=alpha  # correction
)

**Question.** Quel est l'objet retourné par `MapieRegressor` et quelle est sa taille ?

In [ ]:
print(type(y_preds_non_nested), len(y_preds_non_nested))  # correction

**Exercice.** Calculons la RMSE sur le jeu de test, puis la largeur moyenne des intervalles de prédiction, et le taux de couverture. 

In [ ]:
score_non_nested = mean_squared_error(
    y_test, y_preds_non_nested[0], squared=False  # correction
)
widths_non_nested = (
    y_preds_non_nested[1][:, 1, 0] - y_preds_non_nested[1][:, 0, 0]  # correction
)
coverage_non_nested = coverage_score(
    y_test, y_preds_non_nested[1][:, 0, 0], y_preds_non_nested[1][:, 1, 0]  # correction
)

## 3. Validation croisée imbriquée

**Exercice.** Implémentons cette fois-ci directement l'objet `RandomizedSearchCV` dans `MapieRegressor` afin d'effectuer l'étude de paramètres par validation croisée pour chaque modèle "perturbé" servant à calculer les résidus dans MAPIE.

In [ ]:
# Nested approach with the CV+ strategy using the Random Forest model.
cv_obj = RandomizedSearchCV(
    rf_model,  # correction
    param_distributions=rf_params,  # correction
    n_iter=n_iter,  # correction
    cv=cv,  # correction
    scoring="neg_root_mean_squared_error",
    verbose=0,
    random_state=random_state,
    n_jobs=-1,
)
mapie_nested = MapieRegressor(
    cv_obj,  # correction
    method="plus",  # correction
    cv=cv,  # correction
    ensemble=True
)
mapie_nested.fit(X_train, y_train);

**Exercice.** Calculons les intervalles de prédiction sur le jeu de test.

In [ ]:
y_preds_nested = mapie_nested.predict(
    X_test, alpha=alpha  # correction
)

**Exercice.** Calculons la RMSE sur le jeu de test, puis la largeur moyenne des intervalles de prédiction, et le taux de couverture. 

In [ ]:
score_nested = mean_squared_error(
    y_test, y_preds_nested[0], squared=False  # correction
)
coverage_nested = coverage_score(
    y_test, y_preds_nested[1][:, 0, 0], y_preds_nested[1][:, 1, 0]  # correction
)
widths_nested = (
    y_preds_nested[1][:, 1, 0] - y_preds_nested[1][:, 0, 0]  # correction
)
width_diff = (widths_non_nested - widths_nested)/widths_non_nested

## 4. Visualisation des résultats

Comparons les scores et les taux de couverture obtenus avec les deux approches.

In [ ]:
# Print scores and effective coverages.
print(
    "Scores and effective coverages for the CV+ strategy using the "
    "Random Forest model."
)
print(
    "Score on the test set for the non-nested and nested CV approaches: ",
    f"{score_non_nested: .3f}, {score_nested: .3f}"
)
print(
    "Effective coverage on the test set for the non-nested "
    "and nested CV approaches: ",
    f"{coverage_non_nested: .3f}, {coverage_nested: .3f}"
)

Comparons les largeurs des intervalles de prédiction obtenus avec les deux méthodes.

In [ ]:
fig = go.Figure()
min_width = np.min([widths_nested, widths_non_nested])
max_width = np.max([widths_nested, widths_non_nested])
# data
fig.add_trace(go.Scatter(
    name="data",
    x=widths_nested,
    y=widths_non_nested,
    mode="markers",
    marker=dict(color="#1f77b4")
))
# linear curve
fig.add_trace(go.Scatter(
    name="predictions",
    x=[0, 100],
    y=[0, 100],
    mode="lines",
    line=dict(color="#ff7f0e", dash='solid')
))
fig.update_xaxes(range=[14.5, 16.])
fig.update_yaxes(range=[14.5, 16.])
fig.update_layout(
    autosize=False,
    width=600,
    height=600,
    xaxis_title="Prediction interval width using the nested CV approach",
    yaxis_title="Prediction interval width using the non-nested CV approach",
    hovermode="x",
    showlegend=False
)
fig.show()

Visualisons la distribution des différences de largeur entre les deux méthodes.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(
    x=width_diff,
    nbinsx=20,
    marker_color="#1f77b4"
    )
)
# linear curve
fig.add_trace(go.Scatter(
    name="predictions",
    x=[0, 0],
    y=[0, np.histogram(width_diff, bins=20)[0].max()],
    mode="lines",
    line=dict(color="#ff7f0e", dash='solid')
))
fig.update_layout(
    autosize=False,
    width=600,
    height=500,
    xaxis_title="[width(non-nested CV) - width(nested CV)] / width(non-nested CV)",
    yaxis_title="Counts",
    hovermode="x",
    showlegend=False
)
fig.show()